## Preparation for building VR model in Unity for astronomy data
### Convert fits file to binary raw file
method is from "3D visualization of astronomy data cubes using immersive displays"

by Gilles Ferrand, Jayanne English, Pourang Irani

ref: https://arxiv.org/abs/1607.08874

### Try l1448_13co data first

In [21]:
from astropy.io import fits

hdulist = fits.open('l1448_13co.fits')
cube = hdulist[0].data
# And to write the cube to disk as 8-bit integers in a binary file (as expected by my loader on Unity's side):
cube_8bit = ((cube-cube.min())/(cube.max()-cube.min())*255).astype('uint8')
cube_8bit

In [32]:
# cube size needs to be the same on width, height, and depth - NOT necessary

# unity has non-power of two

import numpy as np
# npad is a tuple of (n_before, n_after) for each dimension
b = np.pad(cube_8bit, pad_width=((0, 11), (0,23), (0,23)), mode='constant', constant_values=0)
b.transpose().tofile('l1448co.raw')
b.shape

(64, 128, 128)

### Convert 3D dust map
centered on the Aquila South cloud, cut out distance < 1kpc data, voxel value is magnitudes of AV converted from Bayestar reddenings, as following the similar procedures of plotting_the_dust_map

In [1]:
from dustmaps.config import config
config['data_dir'] = '/Users/penny/Works/bayestar_dustmap'

import dustmaps.bayestar
dustmaps.bayestar.fetch()

File exists. Not overwriting.


In [5]:
from __future__ import print_function
from astropy.coordinates import SkyCoord
from dustmaps.bayestar import BayestarQuery
import numpy as np
import astropy.units as units

bayestar = BayestarQuery(max_samples=1)

def get_ebv_cube():
    # set up a grid of coordinates to plot, centered on the Aquila South cloud
    l0, b0 = (37., -16.)
    l = np.arange(l0 - 5., l0 + 5., 0.05)
    b = np.arange(b0 - 5., b0 + 5., 0.05)
    l, b = np.meshgrid(l, b)
    coords = SkyCoord(l*units.deg, b*units.deg, frame='galactic')
    
    # TODO: why in matplotlib the value is squared?
#     np.sqrt(Av)[::,::-1]

    ebv_cube = bayestar(coords, mode='median')
    
    print(ebv_cube.shape)
    return ebv_cube

ebv_cube = get_ebv_cube()

(200, 200, 31)


In [6]:
np.nanmax(ebv_cube)

0.77891999

In [14]:
# And to write the cube to disk as 8-bit integers in a binary file (as expected by my loader on Unity's side):
ebv_cube_8bit = ((ebv_cube-ebv_cube.min())/(ebv_cube.max()-ebv_cube.min())*255).astype('uint8')
ebv_cube_8bit
import numpy as np
# npad is a tuple of (n_before, n_after) for each dimension
ebv = np.pad(ebv_cube_8bit, pad_width=((0, 56), (0,56), (0,33)), mode='constant', constant_values=0)
ebv.transpose().tofile('dustmap.raw')
ebv.shape

(256, 256, 64)